# Crash Report Jan 18, 2016

Version 2016.06.00.8



In [2]:
"""
General import statements
"""
import crash_analysis
import pandas as pd

In [3]:
"""
Parse zipfiles into dataframe
"""
from crash_analysis.parser import extract_zipfiles, xmldocs_to_dataframe
zipfile_location = 'C:\\CrashReports\\'

extract_zipfiles(zipfile_location)
df = xmldocs_to_dataframe(zipfile_location)

df.drop_duplicates(inplace=True)

In [3]:
"""
Save/load dataframe to/from CSV
"""
new_data = False
cache_location = 'src/data/2016040014.csv'

if new_data:
    df.to_csv(cache_location, encoding='utf-8')
else:
    df = crash_analysis.read_csv(cache_location)

In [4]:
"""
Get metadata about dataframe
"""
col_names = crash_analysis.get_columns(df)
versions = df['AppVersion'].value_counts()
num_crashes = len(df)

print(col_names)
print(versions)
print(num_crashes)

['Unnamed: 0', 'ACCDT_Field', 'Active_ClientFileName', 'Active_Field', 'Active_Form', 'Active_FormsetID', 'Active_FormsetVersion', 'AppName', 'AppVersion', 'BasWin16.INI', 'Batch_ClientFileName', 'CrashGUID', 'CrashRpt', 'Current_Calcsection', 'CustNum', 'CustomProps', 'DataFileCount', 'ExceptionAddress', 'ExceptionCode', 'ExceptionModule', 'ExceptionModuleBase', 'ExceptionModuleVersion', 'ExceptionType', 'FileList', 'FormsPrinter', 'GUIResourceCount', 'GeoLocation', 'ImageName', 'InnerException', 'InstallType', 'Last_Calcsection', 'ManagedException', 'ManagedException.txt', 'MemoryUsageKbytes', 'Message', 'OSIs64Bit', 'OpenHandleCount', 'OperatingSystem', 'ProWin16.INI', 'ProblemDescription', 'Source', 'StackTrace', 'SystemTimeUTC', 'TimeStamp', 'WorkStationName', 'WorkStationType', 'crashdump.dmp', 'crashrpt.xml']
2016040014    5085
Name: AppVersion, dtype: int64
5085


In [6]:
"""
Customer Description Analysis
"""
from crash_analysis.analysis import remove_empty, get_column, stem_frequency
customer_desc = remove_empty(get_column(df, 'ProblemDescription'))

stem_frequency(customer_desc)
print

total words: 7650
instal     	:  463 	 [['installing'], ['installing'], ['installing'], ['installed'], ['install'], ['installing']]
open       	:  349 	 [['opening'], ['opening'], ['opening'], ['opening'], ['opened'], ['opening']]
2016       	:  318 	 [['2016'], ['2016'], ['2016'], ['2016'], ['2016'], ['2016']]
program    	:  254 	 [['program'], ['program'], ['program'], ['program'], ['program'], ['program']]
client     	:  244 	 [['clients'], ['clients'], ['client'], ['client'], ['client'], ['client']]
updat      	:  230 	 [['updating'], ['updates'], ['updates'], ['updates'], ['updates'], ['updates']]
proseri    	:  217 	 [['proseries'], ['proseries'], ['proseries'], ['proseries'], ['proseries'], ['proseries']]
file       	:  217 	 [['file'], ['files'], ['file'], ['file'], ['file'], ['file']]
tri        	:  191 	 [['trying'], ['trying'], ['trying'], ['trying'], ['trying'], ['trying']]
download   	:  166 	 [['downloading'], ['downloading'], ['downloading'], ['downloading'], ['downloadi

In [16]:
"""
Frequency of error messages and stack traces
"""

from crash_analysis import filter_dataframe


def get_value_from_hist(n, field='Message'):
    field_hist = df[field].value_counts()
    assert n < len(field_hist)
    return field_hist.keys()[n], field_hist.values[n]


def print_parings(n, field1='Message', groupings=['AppName', 'InstallType','OperatingSystem']):
    top_key, count = get_value_from_hist(n, field1)
    df_filtered = filter_dataframe(df, **{field1: top_key})
    
    if type(groupings) is type(list()):
        field2_hist = df_filtered.groupby(groupings)['CustNum'].nunique()
    else:
        field2_hist = df_filtered[groupings].value_counts()


    total = sum(field2_hist)

    print("Query {0} \t count: {1} \t {2} ({3}): {4}".format(n+1, count, field1, total, top_key.encode('ascii', 'ignore')))
    
    print(field2_hist)
    print('\n')

# [print_parings(n, field1='ExceptionModuleBase', groupings=['ExceptionAddress', 'ProblemDescription']) for n in range(1, 5)]
# [print_parings(n, groupings='ProblemDescription') for n in [1]]
# print_parings(1, groupings=['CustNum', 'CrashGUID'])
# df['ExceptionAddress'].value_counts()
df[df['CustNum'] == '0423009689']['Message'].value_counts()

Method not found: 'Int32 System.Runtime.InteropServices.Marshal.SizeOf(!!0)'.    13
Name: Message, dtype: int64

In [8]:
df[df['ExceptionAddress'].fillna(' ').str.contains('0x77852a68')].head()

,Unnamed: 0,ACCDT_Field,Active_ClientFileName,Active_Field,Active_Form,Active_FormsetID,Active_FormsetVersion,AppName,AppVersion,BasWin16.INI,...,ProWin16.INI,ProblemDescription,Source,StackTrace,SystemTimeUTC,TimeStamp,WorkStationName,WorkStationType,crashdump.dmp,crashrpt.xml


In [79]:
""" Customer Descriptions """

from crash_analysis import remove_empty
# print(one_cust.head())
pd.set_option('display.max_colwidth', 250)
print(sum(remove_empty(df['ProblemDescription']).value_counts()))

35


In [80]:
"""
Associate by keyterm
"""
from crash_analysis.analysis import associate_by_keyterms

associate_by_keyterms(df, 'ProblemDescription', 'Message', min_count=2)
print

Message by Keyterm
keyterm: program
Method not found: 'Void Intuit.Application.ProSeries.OptionSetupWizardAPI.DataModels.AdditionalInfo..ctor(System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String)'.    2
Method not found: 'Int32 System.Runtime.InteropServices.Marshal.SizeOf(!!0)'.                                                                                                                                                                                                                      1
External component has thrown an exception.                                                                                                                                                                                                                                                        1
Name: Message, dtype: int64

keyterm: open
External component has thrown an exception